In [1]:
!cat /proc/cpuinfo

# cpu가 Intel(R) Xeon(R) CPU @ 2.30GHz이어야함

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [2]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b BM
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 189 (delta 107), reused 102 (delta 20), pack-reused 0
Receiving objects: 100% (189/189), 2.79 MiB | 11.94 MiB/s, done.
Resolving deltas: 100% (107/107), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 9.1 MB/s 
     |████████████████████████████████| 282 kB 63.3 MB/s 
     |████████████████████████████████| 2.8 MB 67.6 MB/s 
     |████████████████████████████████| 919 kB 57.1 MB/s 
     |████████████████████████████████| 1.2 MB 63.1 MB/s 
     |████████████████████████████████| 302 kB 79.0 MB/s 
     |████████████████████████████████| 243 kB 72.2 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 119 kB 77.0 MB/s 
     |████████████████████████████████| 895 kB 75.2 MB/s 
     |██████████████████████████

# 모델 불러오기

각자 알아서 불러올 것

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"
# /content/drive/MyDrive/Colab Notebooks/DistilKoBERT/checkpoint-15000

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/DistilKoBERT")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", model_max_length=512)

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [6]:
# 테스트 데이터

test_df = pd.read_csv("data/test.csv")
test_data = test_df["content"]
test_label = test_df["labels"]

speed_test_data = test_data.head(50).tolist()

## 스피드 테스트

In [7]:
model.to("cpu")
model.eval()
print()

In [8]:
%%timeit

with torch.no_grad():
    tokens = tokenizer(speed_test_data, padding="max_length", truncation=True, return_tensors="pt")
    del tokens["token_type_ids"]
    _ = model(**tokens)

1 loop, best of 5: 19 s per loop


## F1 스코어

In [9]:
from data_loader import get_data_loaders


_, _, test_dataset = get_data_loaders(tokenizer, return_loader=False, use_imbalanced=False, device="cuda")

Using custom data configuration default-fa9241eab75b66a6


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-fa9241eab75b66a6/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

In [10]:
from torch.utils.data import DataLoader

test_dataset = test_dataset.remove_columns("token_type_ids")
test_loader = DataLoader(test_dataset)

In [11]:
from torchmetrics import F1
from tqdm.auto import tqdm


f1_score = F1(num_classes=3, average="macro").cuda()
# f1
model.cuda()
model.eval()

with torch.no_grad():
    for batch in tqdm(test_loader):
        labels = batch.pop("labels")
        output = model(**batch)
        logits = output.logits
        pred = torch.argmax(logits, dim=1)
        f1_score(pred, labels)

score = f1_score.compute()
print(score.item())

  0%|          | 0/5000 [00:00<?, ?it/s]

0.31846100091934204


In [35]:
dataset = test_dataset
df =dataset.to_pandas()
df["labels"].value_counts()

0    4317
1     581
2     102
Name: labels, dtype: int64